In [7]:
def readme():
    print(
    """
    PROJECT TITLE:3D POINT CLOUD COMPRESSION OF KITTI DATASET \n
    Authors: Canderle Filippo, Rossi Cecilia \n
    emails: filippo.canderle@studenti.unipd.it cecilia.rossi.3@studenti.unipd.it
    """)
readme()


    PROJECT TITLE:3D POINT CLOUD COMPRESSION OF KITTI DATASET 

    Authors: Canderle Filippo, Rossi Cecilia 

    emails: filippo.canderle@studenti.unipd.it cecilia.rossi.3@studenti.unipd.it
    


IMPORTING DEPENDENCIES

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
#!pip install open3d
import open3d as o3d

PROCESSING DATA FUNCTIONS

In [2]:
def import_data(directory_points, directory_labels, numfiles):
    """
    In this section we get in input the directories where our .npy files are stored and the 
    number of files that we want to import.
    In outpur we have two list with the point clouds and the related labels.
    
    Input: directories where point clouds and label files are stored, number of files.
    Output: lists of point clouds and labels of point clouds
    """
    points=[]
    labels=[] 
    i=0

    for filename in os.listdir(directory_points):
        if i==numfiles: break
        points.append(np.load(os.path.join(directory_points, filename)))
        i=i+1
    i=0    

    for filename in os.listdir(directory_labels):
        if i==numfiles: break
        labels.append(np.load(os.path.join(directory_labels, filename)))   
        i=i+1
    print("Everything was correctly imported.")
    
    return points, labels

In [3]:
def split_point_clouds(points, labels):
    """
    In this function we separately consider each point cloud in the lists and we "split" each element of 
    the point clouds in single elements in two brand new lists.
    
    Input: lists of point clouds and labels of point clouds
    Output: lists of points and labels of points
    """
    point=[]
    label=[]
    for i in range(len(points)):
        for j in range(len(points[i])):
            point.append(points[i][j].tolist())
            label.append(labels[i][j].tolist())
    point = np.array(point)
    label = np.array(label)
    
    print("The splitting was correctly done.")
    return point,label

In [4]:
def segment_points(point, label):
    """
    In this function we divide all the points in coherent classes, using the list of labels.
    The function print out the number of segmented classes
    
    Input: lists of points and labels of points
    Output: list of classes"(matrix)"
    """
    classes=[]
    for el in range(int(point.max())):
        classes.append([])

    for el in range(len(point)):
        classes[int(label[el])].append(point[el])
    counter=0
    for c in classes:
        if c!=[]: 
            counter=counter+1

    classes=[sublist for sublist in classes if sublist]
    print("The number of different classes that we have is:"+str(len(classes)))
    print("The segmentation was correctly done.")
    return classes

In [5]:
def visualize_class(class_to_visualize, classes):
    """
    In the current function we are able to visualize a chosen class, in order to have an idea 
    of what we are dealing with
    
    Input: the index of chosen class and the list of classes
    No Output
    """
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(classes[class_to_visualize])
    o3d.visualization.draw_geometries([pcd])

In [6]:
def nr_elements_class(classes):
    """
    Here we can visualize the number of element for each class
    
    Input: list of classes
    """
    for i in range(len(classes)):
        print(i,len(classes[i]))
        
    print("We correctly got the number of element for each class.")

O3D COMPRESSION FUNCTIONS

In [7]:
def compression_O3d(classes, rate_O3d):
    """
    This function compress in a lossy way the classes, using the open3d methods and a given rate of compression
    
    Input: list of classes and the rate of compression(list of rates)
    Output: the compressed version of list of classes
    """
    pcd_down=[]
    for el in range(len(classes)):
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(classes[el])
        pcd_down.append(pcd.voxel_down_sample(voxel_size=rate_O3d[el]))
    
    print("First compression was correctly performed.")
    return pcd_down

In [8]:
def visualize_compressed_O3d(num , pcd_down):
    """
    In this section we are able to visualize a compress class, after calling compression_O3d()".
    We are also able to print out the number of points after compression.
    
    Input: the compressed version of list of classes and the index of chosen class.
    No Output
    """
    o3d.visualization.draw_geometries([pcd_down[num]])
    print(pcd_down[num])

In [9]:
def save_compressed_O3d(classes,pcd_down):
    """
    With this function we save the compressed and uncompressed classes in .npy format
    
    Input: compressed and uncompressed versions of classes
    """
    for num in range(len(classes)):
        np.save("o3d_algo/compression/Compressed"+str(num)+".npy", np.asarray(pcd_down[num].points)) #Salvo classe compressa
        np.save("o3d_algo/original/Uncompressed"+str(num)+".npy", np.asarray(classes[num])) #Salvo classe non compressa
    print("We correctly saved the files after o3d compression")

In [10]:
def Upload_Visualize_file(path):
    """
    In this function The .npy file can be uploaded and visualized with open3d methods
    
    Input: Path where the .npy is stored
    """
    points_compressed=np.load(path)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_compressed)
    o3d.visualization.draw_geometries([pcd])
    

RANDOM COMPRESSION ALGORITHM FUNCTIONS

In [11]:
def random_compression(Rate, num_class, classes):
    """
    Here we implement a little algorithm that randomlydrop a certain(given) percentage 
    of point from a certain class.
    It returns a list that contains the mantained 3d points
    
    Input: Rate of compression(list of rates), the class that will be compressed and the list of classes
    Output: Compressed(single) class
    """
    num_points=int(Rate[num_class]*len(classes[num_class]))
    drop=np.random.randint(0, len(classes[num_class]), num_points)
    values_mantained=[]

    for el in range(len(classes[num_class])):
        if el not in drop:
            values_mantained.append(classes[num_class][el])
            
    print("Loss: "+str(len(classes[num_class])-len(values_mantained))+" points.")
    print("Random compression algorithm was applied")
    return values_mantained

In [12]:
def visualize_compressed_class(values_mantained):
    """
    In the current function we are able to visualize the compressed class, in order to have an idea 
    of what we are dealing with
    
    Input: the index of chosen class and the list of classes
    No Output
    """
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(values_mantained)
    o3d.visualization.draw_geometries([pcd])

In [13]:
def save_compressed_alg(values_mantained, classes, num_class):
    """
    We basically save the compressed and uncompressed version of the class in two .npy files
    
    Input: Compressed(single) class, the uncompressed version and the related index.
    No Output
    """
    np.save("random_alg/compression/compressed_class"+str(num_class)+".npy", values_mantained)
    np.save("random_alg/original/uncompressed_class"+str(num_class)+".npy", classes[num_class])
    print("We correctly saved the files after random algorithm")

DRACO CODEC COMPRESSION FUNCTIONS

In [14]:
def setup_draco():
    """
    The needed setup of draco repo on your own pc if you haven't do it yet.
    
    No input or Output needed
    """
    !git clone https://github.com/google/draco
    !mkdir compression_draco
    !cd compression_draco
    !cmake draco
    !make

In [15]:
def visualize_comp_draco(path):
    """
    Here we are able to visualize a compressed point cloud with draco. 
    
    Input: path of file where the .ply file is stored
    No Output
    """
    pcd = o3d.io.read_point_cloud(path)
    o3d.visualization.draw_geometries([pcd])

In [16]:
def create_classes_draco(classes):
    """
    In this function we create the uncompressed version of our classes as a .obj files, that can be read
    by the draco codec's scripts
    
    Input: list of classes
    No output
    """
    for el in range(len(classes)):
        with open("ply_classes/class"+str(el)+".obj", "w") as f:
            for i, p in enumerate(classes[el]):
                f.write("v {} {} {}\n".format(p[0], p[1], p[2]))
    print("We created the .obj files for draco compression")

DISPLAY RESULTS FUNCTION

In [17]:
def displayCompressionRatio(classes):
    """
    For every CLASS of objects, let's evaluate how much the space in memory of the data file 
    changes after every type of compression, with respect to the original data
    
    Input: List of classes
    No Output
    """
    for n_class in range(len(classes)):
        print('CLASS '+str(n_class))
        nfile_original = "o3d_algo/original/Uncompressed"+str(n_class)+".npy" 
        #I just consider this because the file has the same dimension before every type of compresion
        
        nfile_compressed_od3 = "o3d_algo/compression/Compressed"+str(n_class)+".npy"
        nfile_compressed_random = "random_alg/compression/compressed_class"+str(n_class)+".npy"
        
        nfile_original_draco="/Users/filippo/Desktop/ply/class"+str(n_class)+'.obj'
        nfile_compressed_draco = "/Users/filippo/Desktop/ply/dec_c"+str(n_class)+'.ply'
        
        stats_info_orig = os.stat(nfile_original)
        stats_info_od3 = os.stat(nfile_compressed_od3)
        stats_info_random = os.stat(nfile_compressed_random)
        stats_info_orig_draco = os.stat(nfile_original_draco)
        stats_info_draco = os.stat(nfile_compressed_draco)
        
        print('Dimension of the original data of the class '+str(n_class)+'(in bytes):\n', stats_info_orig.st_size)
        print("\n")
        print('Dimension of the data of the class '+str(n_class)+' after the compression with O3D COMPRESSION (in bytes):\n', stats_info_od3.st_size)
        print('O3D Compression Ratio: '+str( stats_info_od3.st_size/stats_info_orig.st_size))
        print("\n")
        print('Dimension of the data of the class '+str(n_class)+' after the compression with RANDOM COMPRESSION ALGORITHM (in bytes):\n', stats_info_random.st_size)
        print('RANDOM Compression Ratio: '+str( stats_info_random.st_size/stats_info_orig.st_size))
        print("\n")
        print('Dimension of the .obj file of the class '+str(n_class)+'(in bytes):\n', stats_info_orig_draco.st_size)
        print('Dimension of the data of the class '+str(n_class)+' after the compression with DRACO CODEC COMPRESSION (in bytes):\n', stats_info_draco.st_size,'\n')
        print('DRACO Compression Ratio: '+str(stats_info_draco.st_size/stats_info_orig_draco.st_size))
        
        print("\n\n\n")
    

MAIN FUNCTIONS: From here we are able to call the previous (nested) functions.

In [18]:
def processingdata(class_to_visualize):
    """
    This is the main of our work, from here we can call all the needed functions in order to do preprocessing.
    
    INPUTS required: class to visualize, that is a variable that allow us to visualize a class after preprocessing
    Output: List of classes
    
    """
    directory_points = "velodyne/velodyne"
    directory_labels = "velodyne/labels"
    numfiles=100
    #DATA HANDLING
    points, labels=import_data(directory_points, directory_labels, numfiles)
    
    point,label=split_point_clouds(points, labels)
    
    classes=segment_points(point, label)
    
    nclass = class_to_visualize
    visualize_class(class_to_visualize, classes)
    
    nr_elements_class(classes)
    
    return classes


In [19]:
def compression(classes, tipo, nclass, r=None):
    """
    Here we perform compression calling the selected type of compression
    
    Input: list of classes, type of selected method to apply, class to be visualized in order to have an idea
    and r, that is an optional argument if we want to manually set the rates with Manual_Rate_selection()
    
    No Output
    """
    #FIRST COMPRESSION: O3D Methods
    class_to_visualize=nclass
    if tipo==1:
        print("YOU SELECTED O3D LIBRARY")
        rate_O3d=r
        
        if rate_O3d==None:
            rate_O3d=float(input("Set the rate of compression: ")) #0.2 was the number that we used here
            rate_O3d=[rate_O3d for i in range(len(classes))]
        
        path="compression/Compressed"+str(nclass)+".npy"
    
        pcd_down=compression_O3d(classes, rate_O3d)
        visualize_compressed_O3d(class_to_visualize , pcd_down)
        save_compressed_O3d(classes,pcd_down)
        Upload_Visualize_file(path)
        print("STATUS: Completed")
        
    #SECOND COMPRESSION: A little algorithm(that can be iteratively called by adding a for loop)
    elif tipo==2:
        print("YOU SELECTED RANDOM ALGORITHM")
        
        Rate=r
        
        if Rate==None:
            Rate=float(input("Set the rate of compression: "))#0.90 was the number that we used here
            Rate=[Rate for i in range(len(classes))]
        for c in range(len(classes)):
            values_mantained=random_compression(Rate, c, classes)
            visualize_class(nclass, classes) 
            visualize_compressed_class(values_mantained) 
        
            save_compressed_alg(values_mantained, classes, c)
        print("STATUS: Completed")
        
    #THIRD COMPRESSION: Using draco codec by google
    elif tipo==3:
        print("YOU SELECTED DRACO CODEC")
        create_classes_draco(classes)
        #setup_draco() #This function needs to be called only once.
        """
        for i in range(len(classes)):
            get_ipython().system('draco_encoder -i Desktop/ply/class'+str(el)+'.obj -o Desktop/ply/comp_c'+str(el)+'.drc')
            get_ipython().system('draco_decoder -i Desktop/ply/comp_c'+str(el)+'.drc -o Desktop/ply/dec_c'+str(el)+'.ply')
            
        NOTE: This part is commented because in our implementation we called these commands from shell, but 
        we decided to add them in this jupyter for completeness.
        """
    
        path='/Users/filippo/Desktop/ply_classes/dec_c'+str(nclass)+'.ply'
        visualize_comp_draco(path)
        print("STATUS: Completed")
    else:
        raise Exception("Sorry, this number is not allowed")

In [20]:
def show_methods():
    """
    We can print the different avaiable methods that we can select.
    """
    print("AVAIABLE COMPRESSIONS FOR 3D POINT CLOUD")
    print("1-Using open3d library")
    print("2-Using a little algorithm")
    print("3-Using draco codec(You'll need to use shell) ")

In [21]:
def Manual_Rate_selection(classes, nclass):
    """
    This function is implemented in order to allow us to manually select all the compression rates for every class
    and for O3d method and random algorithm
    
    INPUT: list of classes and class to visualize in order to make sure to do a correct preprocessing
    NO OUTPUT
    
    """
    r1=[]
    r2=[]
    
    for el in range(len(classes)):
        visualize_class(el, classes)
        r1.append(float(input("Select the Rate for O3D method: ")))
        r2.append(float(input("Select the Rate for Random method: ")))
        
    compression(classes, 1, nclass, r1)
    compression(classes, 2, nclass, r2)

MAIN: Here we preprocess datas and prepare the different classes

In [24]:
print('PROCESSING DATA')
nclass = int(input("Choose the class to Upload and visualize: "))
classes = processingdata(nclass)
numclasses = len(classes)

PROCESSING DATA
Choose the class to Upload and visualize: 6
Everything was correctly imported.
The splitting was correctly done.
The number of different classes that we have is:20
The segmentation was correctly done.
0 444578
1 376322
2 7304
3 4328
4 13397
5 19526
6 6401
7 3221
8 333
9 1134266
10 49133
11 781627
12 20035
13 1073767
14 83591
15 2834524
16 90597
17 1080155
18 29340
19 8347
We correctly got the number of element for each class.


If we want to use a certain fixed rate for every class we can use the following:

In [ ]:
#To be used with O3D: type_comp=1
show_methods()
type_comp=int(input("Select the type of compression: "))
compression(classes, type_comp, nclass)

In [ ]:
#To be used with Random Algorithm: type_comp=2
show_methods()
type_comp=int(input("Select the type of compression: "))
compression(classes, type_comp, nclass)

In [ ]:
#To be used with Draco Codec: type_comp=3
show_methods()
type_comp=int(input("Select the type of compression: "))
compression(classes, type_comp, nclass)

Alternatively, if we want to manually set the rate for every class we can use the following:

In [ ]:
Manual_Rate_selection(classes, nclass)

And finally we can evaluate the performances of our work easily calling this function:

In [25]:
displayCompressionRatio(classes)

CLASS 0
Dimension of the original data of the class 0(in bytes):
 10670000


Dimension of the data of the class 0 after the compression with O3D COMPRESSION (in bytes):
 6667256
O3D Compression Ratio: 0.6248599812558575


Dimension of the data of the class 0 after the compression with RANDOM COMPRESSION ALGORITHM (in bytes):
 4332344
RANDOM Compression Ratio: 0.40603036551077787


Dimension of the .obj file of the class 0(in bytes):
 49789098
Dimension of the data of the class 0 after the compression with DRACO CODEC COMPRESSION (in bytes):
 10096416 

DRACO Compression Ratio: 0.20278366962984548




CLASS 1
Dimension of the original data of the class 1(in bytes):
 9031856


Dimension of the data of the class 1 after the compression with O3D COMPRESSION (in bytes):
 1832360
O3D Compression Ratio: 0.20287745951662647


Dimension of the data of the class 1 after the compression with RANDOM COMPRESSION ALGORITHM (in bytes):
 3668048
RANDOM Compression Ratio: 0.40612339257844676


Dimensio